# 대스타

[참고]https://datacook.tistory.com/59

### 환경 설정

In [1]:
import tensorflow as tf

tf.__version__

'2.8.2'

In [2]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import keras

keras.__version__

'2.8.0'

# 라이브러리 선언

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model,Sequential
import zipfile

import gdown

### Data Download

In [10]:
google_path = 'https://drive.google.com/uc?id='
file_id = '1QE56cgOZcwV4r4CzNEs_BdoIp2zLl2IA'
output_name = 'train.zip'
gdown.download(google_path+file_id,output_name)

#https://lapina.tistory.com/140

Downloading...
From: https://drive.google.com/uc?id=1QE56cgOZcwV4r4CzNEs_BdoIp2zLl2IA
To: /content/train.zip
100%|██████████| 1.37G/1.37G [00:24<00:00, 55.6MB/s]


'train.zip'

In [11]:
input_path = './train.zip'
output_path = './data'

In [12]:
zip_data = zipfile.ZipFile(input_path)
zip_data.extractall(output_path)

In [13]:
train_path = './data/re_train'

In [14]:
# 예측할 클래스 수
classes = 309

# Input으로 사용될 크기와 채널수
height = 256
width = 256
channels = 3

In [23]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1)
image_data_train = image_generator.flow_from_directory(train_path,subset='training')
image_data_test = image_generator.flow_from_directory(train_path,subset='validation')

Found 39025 images belonging to 309 classes.
Found 4207 images belonging to 309 classes.


### Model Load

In [24]:
resnetv2 = tf.keras.applications.ResNet50V2(include_top=False,input_shape=(height,width,channels))

In [25]:
resnetv2.trainable=False

In [26]:
model = Sequential([
                 resnetv2,
                 Dense(512,activation='relu'),
                 BatchNormalization(),
                 GlobalAveragePooling2D(),
                 Dense(classes,activation='softmax')
])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 dense_2 (Dense)             (None, 8, 8, 512)         1049088   
                                                                 
 batch_normalization_1 (Batc  (None, 8, 8, 512)        2048      
 hNormalization)                                                 
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 309)               158517    
                                                                 
Total params: 24,774,453
Trainable params: 1,208,629
N

In [28]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [29]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [30]:
lrs = tf.keras.callbacks.LearningRateScheduler(scheduler)

### Model fit

In [31]:
model.fit(image_data_train,batch_size=64,epochs=10,callbacks=[lrs],validation_data=(image_data_test),
          validation_steps =image_data_test.samples/image_data_test.batch_size)

Epoch 1/10
1220/1220 [==============================] - 184s 139ms/step - loss: 4.0055 - accuracy: 0.1600 - val_loss: 3.5717 - val_accuracy: 0.2318 - lr: 0.0010
Epoch 2/10
1220/1220 [==============================] - 170s 140ms/step - loss: 2.5881 - accuracy: 0.4078 - val_loss: 2.8637 - val_accuracy: 0.3449 - lr: 0.0010
Epoch 3/10
1220/1220 [==============================] - 171s 140ms/step - loss: 1.8654 - accuracy: 0.5694 - val_loss: 2.0977 - val_accuracy: 0.5165 - lr: 9.0484e-04
Epoch 4/10
1220/1220 [==============================] - 170s 139ms/step - loss: 1.4504 - accuracy: 0.6645 - val_loss: 1.7992 - val_accuracy: 0.5712 - lr: 8.1873e-04
Epoch 5/10
1220/1220 [==============================] - 170s 139ms/step - loss: 1.1970 - accuracy: 0.7212 - val_loss: 1.7036 - val_accuracy: 0.5876 - lr: 7.4082e-04
Epoch 6/10
1220/1220 [==============================] - 170s 140ms/step - loss: 1.0187 - accuracy: 0.7607 - val_loss: 1.5172 - val_accuracy: 0.6263 - lr: 6.7032e-04
Epoch 7/10
1220/12